In [1]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Cursor
from pprint import pprint
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Sara
[nltk_data]     Palacios\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
consumer_key = 'X8ZPbUxvrjaFaWF4zh56HQ7Dk'
consumer_secret = 'wcmzR7DnsvEZICxYIMAcdDwR1vp4NWHqJ9QGsG6GpT4RJkuwzu'
access_token = '3407979129-llI6bmu6J7aQclstfHmzww86CgxUdPIJf93ELh4'
access_secret = 'PkoRudJ87ZjHBUCqGDvFfjCjKflVw61LIkzlzEi3NH642'

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

In [4]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    #Para probar la conexion, si sirven o no las llaves
    print(status.text)

TweepError: Twitter error response: status code = 401

In [5]:
from tweepy import Stream
from tweepy.streaming import StreamListener

In [ ]:
class MyListener(StreamListener):
 #para probar la libreria, no se corre 
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True
 
twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#python'])

In [6]:
account_list = ["Alpina"]
#Se agrega la cuenta que se va a analizar
for target in account_list:
    item = api.get_user(target)
    print("name: " + item.name)
    print("screen_name: " + item.screen_name)
    print("description: " + item.description)
    print("statuses_count: " + str(item.statuses_count))
    print("friends_count: " + str(item.friends_count))
    print("followers_count: " + str(item.followers_count))
    tweets = item.statuses_count
    for status in Cursor(api.user_timeline, id=target).items(1):
        pprint(status.text)

TweepError: [{'code': 89, 'message': 'Invalid or expired token.'}]

In [ ]:
import json
#Para analizar la cantidad de tweets a evaluar
#El query se cambia con el nombre de la cuenta
max_tweets=10000
query='GoogleColombia'
searched_tweets = [status._json for status in tweepy.Cursor(api.user_timeline, id=query).items(max_tweets)]
json_strings = [json.dumps(json_obj) for json_obj in searched_tweets]  

In [ ]:
#para guardar los tweets en el archivo
#se cambia el nombre del archivo .json
with open('GoogleColombia.json', 'a', encoding='utf8') as outfile:
    for json_obj in searched_tweets:
        json.dump(json_obj, outfile)
        outfile.write('\n')
    #[json.dumps(json_obj,outfile) for json_obj in searched_tweets] 

In [ ]:
pprint(searched_tweets)

In [ ]:
#Preprocessing with Emoticons
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [ ]:
#Reading | Tokenizing From a File
#cambiar nombre 

with open('GoogleColombia.json', 'r', encoding='utf8') as f:
    for line in f:
        tweet = json.loads(line)
        #tokens = word_tokenize(tweet['text'])
        tokens = preprocess(tweet['text']) #para tener en cuenta los emojis
        
        print(tokens) # pretty-print

In [ ]:
#Counting Terms
import operator 
import json
from collections import Counter

In [ ]:
#Removing Stop Words (spanish)
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import bigrams 
from collections import defaultdict
import string
set(stopwords.words('spanish'))

In [ ]:
#Se excluyen de las palabras los signos de puntuacion y las stopwords
punctuation = list(string.punctuation)
stop = stopwords.words('spanish') + punctuation + ['rt', 'via', '¡','…','¿'] + ['http', 'https', 'co']
print(stop)

In [ ]:
#se vuelve a cambiar el nombre del archivo
with open('GoogleColombia.json', 'r', encoding='utf8') as f:
    count_all = Counter()
    count_stop = Counter()
    count_hash = Counter()
    count_bigrams = Counter()
    for line in f:
        tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'],True)]
        terms_stop = [term for term in preprocess(tweet['text'],True) if term not in stop]
        # Count hashtags only
        terms_hash = [term for term in preprocess(tweet['text']) if term.startswith('#')]
        # Count bigrams
        terms_bigram = bigrams(terms_stop)
        # Update the counter
        count_all.update(terms_all)
        count_stop.update(terms_stop)
        count_hash.update(terms_hash)
        count_bigrams.update(terms_bigram)
    # Print the first 5 most frequent words
    print(count_all.most_common(5)) # analiza el texto del tweet completo
    print(count_stop.most_common(5)) # analiza despues del preprocesamiento
    print(count_hash.most_common(5)) # analiza los hashtags
    print(count_bigrams.most_common(5)) #analiza las dos palabras que salen en un mismo tweet

In [ ]:
#Term co-occurrences Analysis
com = defaultdict(lambda : defaultdict(int))
with open('GoogleColombia.json', 'r', encoding='utf8') as f:
    for line in f: 
        tweet = json.loads(line)
        terms_only = [term for term in preprocess(tweet['text'],True) 
                      if term not in stop 
                      and not term.startswith(('#', '@'))]

        # Build co-occurrence matrix
        for i in range(len(terms_only)-1):            
            for j in range(i+1, len(terms_only)):
                w1, w2 = sorted([terms_only[i], terms_only[j]])                
                if w1 != w2:
                    com[w1][w2] += 1
    com_max = []
    # For each term, look for the most common co-occurrent terms
    for t1 in com:
        t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
        for t2, t2_count in t1_max_terms:
            com_max.append(((t1, t2), t2_count))
    # Get the most frequent co-occurrences
    terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
    print(terms_max[:5])

In [ ]:
#Specific Term for Cocurrence
search_word = 'desarrollo' # pass a term for looking into Company Tweet
count_search = Counter()
with open('GoogleColombia.json', 'r', encoding='utf8') as f:
    for line in f:
        tweet = json.loads(line)
        terms_only = [term for term in preprocess(tweet['text'],True) 
                      if term not in stop 
                      and not term.startswith(('#', '@'))]
        if search_word in terms_only:
            count_search.update(terms_only)
print("Co-occurrence for %s:" % search_word)
print(count_search.most_common(20))